In [1]:
import pandas as pd
import numpy as np
import os
import shutil

In [2]:
PERIOD_YEAR = '2014'

In [3]:
df_interactions = pd.read_csv('../data/userid_trackid_timestamp.tsv', sep='\t')
print(df_interactions.shape)
df_interactions.head()

(252984396, 3)


,user_id,track_id,timestamp
0,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:42:38
1,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:38:53
2,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:35:08
3,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:31:23
4,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:27:38


In [4]:
df_interactions['year'] = pd.to_datetime(df_interactions['timestamp']).dt.to_period('Y')
df_interactions['month'] = pd.to_datetime(df_interactions['timestamp']).dt.to_period('M')
df_interactions = df_interactions[df_interactions['year'] == PERIOD_YEAR]
print(df_interactions.shape)
df_interactions.head()

(33710223, 5)


,user_id,track_id,timestamp,year,month
368,24313,u0VSWWDYp8f4jAbF,2014-03-11 23:33:51,2014,2014-03
369,24313,b3YJnGaS1CGIYBHg,2014-03-11 21:53:19,2014,2014-03
370,24313,9aU42QoMlPnoec2y,2014-03-10 22:41:10,2014,2014-03
371,24313,8URFbJvh6yRV8tsp,2014-03-10 22:30:04,2014,2014-03
372,24313,eIdhO4oir0wtKDsc,2014-03-10 20:34:42,2014,2014-03


In [5]:
# binarization: we convert the interactions to binary implicit feedback with a threshold of 2 on the interaction counts (reducing false-positive interactions)
def filter_interactions(listening_history: pd.DataFrame, min_interactions: int = 2, verbose: bool = True):
    lhs_count = listening_history.value_counts(subset=['user_id', 'track_id'])
    lhs_count = lhs_count[lhs_count >= min_interactions]
    listening_history = listening_history.set_index(['user_id', 'track_id']).loc[lhs_count.index]
    listening_history = listening_history.reset_index()
    return listening_history

def delete_duplicates(listening_history: pd.DataFrame):
    initial_shape = listening_history.shape
    listening_history = listening_history.drop_duplicates(subset=['user_id', 'track_id'], keep='first')
    print(f"Deleted {initial_shape[0] - listening_history.shape[0]} duplicates")
    return listening_history

df_interactions = filter_interactions(df_interactions, min_interactions=2)
df_interactions = delete_duplicates(df_interactions)
print(df_interactions.shape)
df_interactions.head()

Deleted 24123684 duplicates
(4463058, 5)


,user_id,track_id,timestamp,year,month
0,117534,hxGBTxhSf7o6b1fz,2014-08-24 19:40:30,2014,2014-08
9791,117534,fSQrzRe2g3gsEMoq,2014-08-24 19:55:29,2014,2014-08
19557,117534,I53KHO4GCKMeUKUN,2014-08-24 19:47:52,2014,2014-08
29295,117534,nd4vejLsgjjhl55B,2014-08-24 19:36:23,2014,2014-08
39030,117534,WR82nGL7vKxitC5Q,2014-08-24 19:51:04,2014,2014-08


In [6]:
# sparsity of dataset
n_users = df_interactions['user_id'].nunique()
n_items = df_interactions['track_id'].nunique()
n_interactions = df_interactions.shape[0]
sparsity = 1 - n_interactions / (n_users * n_items)
sparsity

0.9976135897290934

In [7]:
n_items

37781

In [8]:
n_users

49501

In [9]:
# export
df_interactions.to_csv(f'../data/userid_trackid_timestamp_{PERIOD_YEAR}.tsv', sep='\t', index=False)